In [26]:
import pandas as pd
import numpy as np
#!pip install pygam
import pygam

In [165]:
def ifelse(bool_arg,true,false):
    if(bool_arg):
        return true
    else:
        return false
def optim_kumar(corr_mtx=[], P_in=[],
                        W_in = [0.], R_in = [0.], Z_in =[0.],
                        Wlim = 0, Rlim = 3, Zlim = 0.01,
                        k_alpha = 0.05, z_alpha = 0.05, eps=0.05,
                        penalty_RZ = 0, elastic_RZ = 0.5,Clim=[0],CashLim=pd.NA,
                        algorithm = "NLOPT_LD_SLSQP", controls = list(),
                        S_FN = [], relax = False, browse =False) :
    W_optim = [] #Index for W elements to be optimized in the vector W.in=W in the composing the mathematical formulation
    R_optim = [] #Index for R elements to be optimized in the vector R.in=R in the composing the mathematical formulation
    Z_optim = [] #Index for Z elements to be optimized in the vector C.in=C in the composing the mathematical formulation
    W_indx = 0 #Index for W elements to be optimized in vector passed to optimization routine
    R_indx = 0 #Index for R "
    Z_indx = 0 #Index for Z "
    W_TF = False #Are there W variables to be optimized?
    R_TF = False #Are there R "
    Z_TF = False #Are there Z "
    S_Tf = False #Will there be novation occurring?
    C_TF = False #Only used for forward-prop because we must ensure that w_ast+w_nov=pre-defined value
    n_S = 0 # number of supply functions passed in (should always be 1. >1 if more than one asset to optimize)      
    n_C = 0 # Only used for forward-prop, number of combined sets passed in (should always be 1. >1 if more than one asset to optimize)     
    Cash_TF = False #is there cash constraint in the portfolio
    
    if (pd.isnull(CashLim)):
        pass
    else :
        cash_TF=True
        # The given command concatenates 2 numpy arrays 
        W_in=CashLim+W_in
        R_in=[1.01]+R_in
        Z_in=[1]+Z_in
        if(len(P_in)>0):
            k=[[0.9999,100,1]]
            P_in=k+P_in
        else:
            P_in=[0.9999]
            ## What exactly is the Line 92 doing
                
        """
        Modify the Line Number 92
        The part of adding checks to correlation Matrix is left
        """
        k=np.shape(corr_mtx)
        m=np.zeros(k[1])
        d=np.insert(corr_mtx,[0],m,axis=0)
        l=np.zeros(k[0]+1)
        l=np.transpose(l)
        corr_mtx=np.insert(d,0,l,axis=1)
        corr_mtx[0,0]=1
        
    if(np.count_nonzero(Clim)==len(Clim)):
        C_TF=True
        n_C=len(Clim)
    """
    Which variables correspond to Supply Function (S.FN)
    Note Here:
    The S.optim in R has been actually been replaced by a:b
    """
    if(len(S_FN)>0 and type(ini_list1) is list):
        S_TF = True #supply function has been passed in (which means novation is occurring)
        #S.optim = c((nrow(corr.mtx)+1-length(S_FN)):nrow(corr.mtx)) #correlation matrix is constructed s.t. last elements correspond to novation 
        s=shape(corr_mtx)
        a=s[0]+1-len(S_FN)
        b=s[0]
        W_in[a:b] = 0.01 #just setting a random default value for now for amount novated
        n_S = length(b-a)
        print('n')
    else:
        S_optim=0
    """
    Checks if any element is zero- That basically needs to be optimized
    The three snnipets below give us :
     Which elemnt in W,R,Z to optimize
    """
    if(np.all(W_in)):
        pass
    else:
        if(len(W_in) == 1):
            # We are changing the  notataio here W.optim <- c(1:nrow(corr.mtx))
            c=np.shape(corr_mtx)
            W_optim=c[0]
            W_in=np.zeros(c[0])
        else:
            
            k=np.array(W_in)
            W_optim=np.where(k==0)[0]
            W_optim=W_optim[0]
        W_TF=True
    
    if(np.all(R_in)):
        pass
    else:
        if(len(R_in) == 1):
            # We are changing the  notataio here W.optim <- c(1:nrow(corr.mtx))
            c=np.shape(corr_mtx)
            R_optim=c[0]
            R_in=np.zeros(c[0])
        else:
            
            k_1=np.array(R_in)
            R_optim=np.where(k==0)[0]
            R_optim=R_optim[0]
        R_TF=True
            
    if(np.all(Z_in)):
        pass
    else:
        if(len(Z_in) == 1):
            # We are changing the  notataion here W.optim <- c(1:nrow(corr.mtx))
            c=np.shape(corr_mtx)
            Z_optim=c[0]
            Z_in=np.zeros(c[0])
        else:
            
            k=np.array(Z_in)
            Z_optim=np.where(k==0)[0]
            Z_optim=Z_optim[0]
        Z_TF=True
    #Number of Variables to send to optimizer    
    n_opt = W_optim+R_optim+Z_optim
    #### Create Funtions to interpolate values and gradients of S.FN from MGCV ####
    if(S_Tf):
        def RZ_S_calc(R,Z):
            global R_in,W_optim
            Rc=R_in[0:W_optim]
            Zc=Z_in[0:W_optim]
            if (len(R)>0):
                Rs=Rc-R
                Zs = Zc-Z
                return Rs, Zs
            m=len(Rc)
            if (len (R)==m and len(Z)==m):
                Rs = Rc-R
                Zs = Zc-Z
            elif(np.shape(R)[1] == m and np.shape(R)[1] == m):
                """
                Robustness is assumed here:
                 Rc and Zc are fit to be multiplied
                """
                m1=np.ones([1,np.shape(R)[0]])
                m2=np.ones([1,np.shape(Z)[0]])
                Rs=(Rc*m1).T-R
                Zs=(Zc*m2).T-Z
            return[Rs,Zs]
        
        def S_intrp(R, Z, inpt_adj = True):
            def S_intrp_set(mdls, R, Z):
                S=np.zeros(len(R))
                for a in mdls:
                    """
                    Note we initially Fit the Gam using the following:
                    gam = LogisticGAM().gridsearch(X_train, y_train)
                    mdls is a set of Set of such objects 

                    """
                    t=a.predict([R,Z])
                    S=S+t
                return S
            if(inpt_adj):
                RZ =RZ_S_calc(R, Z)
                R = RZ.R
                Z = RZ.Z
            ## Simple Case
            m=len(S_FN)
            if(m==1):
                if isinstance(pygam.LinearGAM(),S_FN[1]):
                    S=S_intrp_set(S_FN[1],R,Z)
                else:
                    S=S_intrp_set(S_FN[[1]],R,Z)
            if(len(np.shape(R))==1 and len(np.shape(Z))==1):
                if(len(R)==m and len(Z)==m):
                    R=R.T
                    Z=Z.T
                else:
                    R = np.repeat(R, repeats=m, axis=0)
                    Z = np.repeat(Z, repeats=m, axis=0)
            S=np.zeros([shape(R)[0],m])
            for j in range(m):
                """
                S is a array of arrays
                """
                S[:,j]= [S_N[j], R[:, j], Z[:, j]]
            return S
        #Interpolate derivative w.r.t R
        def S_dR_intrp(R, Z, delta = (1e-6)/2, inpt_adj = True):
            S_pre = S_intrp(R-delta, Z, inpt_adj = inpt_adj)
            S_pst = S.intrp(R, Z+delta, inpt_adj = inpt_adj)
            return ((S_pst-S_pre)/(2*delta))
        
        #### Define/Update Vectors and Vector-Derivatives ####
        dWin_dw=np.zeros(len(W_in))
        dWin_dr=np.zeros(len(W_in))
        dWin_dz=np.zeros(len(W_in))
        dRin_dw=nr.zeros(len(R_in))
        dZin_dz=np.zeros(len(Z_in))
        if(W_TF): #If any allocations to be optimized
            dWin_dw[W_optim] = 1

        if(Cash_TF): #if cash constraint
            dWin_dw[1] = -1
  
        if(S_TF): #If supply function provided
            dRin_dr[R_optim] = 1
            dZin_dz[Z_optim] = 1
        """
        Scoping requirement: we would like to change values of variables that are not explicitly passed 
        into the function that is they are declared outside the function, BUT, we DON'T want those
        changes to persist outside the function e.g. suppose W.in = [1,2,3] and inside f() 
        we modify W.in such that W.in = [1,2,4] then that change to W.in should only be visible 
        inside f() outside f() W.in remains W.in=[1,2,3] in R if we want changes to\
        persist outside f() then we do assignment W.in <<- [1,2,4] rather than W.in <- [1,2,4]
        """
        #Parse X.opt, update W.in, R.in, Z.in, dWin.dw, dWin.dr, dWin.dz, dRin.dr, dZin.dz
        def opt_update_WRZ(X):
            
            global W, R, Z, dW_dw, dW_dr, dW_dz, dR_dr, dZ_dz 
            
            if(W_TF):
                W_in[W_optim] = X[W_indx]
                if(Cash_TF):
                    W_in[1] = CashLim-sum(X[:W_indx])
            if(R_TF):
                R_in[R_optim] = X[R_indx]
            if(Z_TF):
                Z_in[Z_optim] = X[Z_indx]
            if (S_TF):
                """
                If supply function has been provided
                indices ending in 'S' indicate that its related to the supply function
                """
                W_in[S_optim] = S_intrp(X[RS_indx], X[ZS_indx])
                dWin_dr[S_optim] = S_dR_intrp(X[RS_indx], X[ZS_indx])
                dWin_dz[S_optim] = S_dZ_intrp(X[RS_indx], X[ZS_indx])
                if(Cash_TF):
                    dWin_dz[1] = -W_in[S_optim]
                    if(W_TF):
                        
                        W_in[1]=W_in[1]-sum(W_in[0:S_optim]*(-X[ZS_indx]))
                    else:
                        W_in[1]=CashLim-sum(W_in[0:S_optim]*(-X[ZS_indx]))
                """
                This Changes the global variables
                """
                W = W_in 
                R = R_in
                Z = Z_in 
                dW_dw = dWin_dw
                dW_dr = dWin_dr
                dW_dz = dWin_dz
                dR_dr = dRin_dr
                dZ_dz = dZin_dz
                return [W, R, Z, dW_dw, dW_dr, dW_dz, dR_dr, dZ_dz]
            
            def obj(params):
                W = params.W
                R = params.R
                Z = params.Z
                #calculate lender profit
                prft = np.multiply((W.T),(np.multiply(P_in,(R-Z))+Z))
                #calculate scalaing factor and variance
                scale_fct = np.sqrt((1-eps)/eps)
                Var = np.multiply(W,np.multiply((R-Z),Std)).T*corr_mtx*np.multiply(W,np.multiply((R-Z),Std))
                return scale_fct*np.sqrt(Var[1,1])-prft[1,1]
                
            def d_obj_dw(params):
                W = params.W
                R = params.R
                Z = params.Z
                dW.dw = params.dW_dw
                dprft_dw = (dW_dw.T)*(np.multiply(P_in,(R-Z))+Z)
                scale_fct = np.sqrt((1-eps)/eps)
                Var = np.multiply(W,np.multiply((R-Z),Std)).T*corr_mtx*np.multiply(W,np.multiply((R-Z),Std))
                dVar_df = 2*(corr_mtx*np.multiply(W,np.multiply((R-Z),Std))) #n x 1
                df_dw = np.multiply(Std,np.multiply((R-Z)*dW_dw))
                #Combine
                dVar_dw = scale_fct*1/2*Var[1,1]^(-1/2)*dVar_df[W_optim]*(df_dw[W_optim]+df_dw[1]*Cash_TF)
                
                return(dVar_dw-dprft_dw)
            
            def d_obj_d(params):
                W = params.W
                R = params.R
                Z = params.Z
                dW_dr = params.dW_dr
                dR_dr = params.dR_dr
                #derivative of profit w.r.t w_as
                dprft_dr = (dW_dr).T*(np.multiply(P_in,(R-Z)+Z))+(W.T)*(np.multiply(P_in,dR_dr))
                ##derivative of variance w.r.t. w_ast
                #calculate terms
                scale_fct = np.sqrt((1-eps)/eps)
                Var = np.multiply(W,np.multiply((R-Z),Std)).T*corr_mtx*np.multiply(W,np.multiply((R-Z),Std))
                dVar_df = 2*(corr_mtx*np.multiply(W,np.multiply((R-Z),Std)))
                df_dw = np.multiply(Std,np.multiply((R-Z)*dW_dr)-np.multiply(W,dR_dr))
                #Combine
                dVar_dr = scale_fct*1/2*(Var[1,1]^(-1/2))*(dVar_df*df_dr)(R_optim)#%*%t(df.dr)
                return(dVar_dr-dprft_dr)
            
            def d_obj_dz(params):
                
                    
                W = params.W
                R = params.R
                Z = params.Z
                dW_dz = params.dW_dz
                dZ_dz = params.dZ_dz
                dprft_dz = (dW_dz).T*(np.multiply(P_in,(R-Z)+Z))+(W.T)*(np.multiply(P_in,dZ_dz))
                ##derivative of variance w.r.t. w_ast
                #calculate terms
                scale_fct = np.sqrt((1-eps)/eps)
                Var = np.multiply(W,np.multiply((R-Z),Std)).T*corr_mtx*np.multiply(W,np.multiply((R-Z),Std))
                dVar_df = 2*(corr_mtx*np.multiply(W,np.multiply((R-Z),Std)))
                df_dz = np.multiply(Std,np.multiply((R-Z)*dW_dz)-np.multiply(W,dZ_dz))
                #Combine
                dVar_dz = scale_fct*1/2*(Var[1,1]^(-1/2))*(dVar_df*df_dz)*(Z_optim)#%*%t(df.dr)
                return(dVar_dz-dprft_dz)
            
            #hessian of profit
            #hessian of profit (not used)
            def f_prft_hsn(params):
                
                W = params.W
                R = params.R
                Z = params.Z
                dprft_dW = np.multiply(P_in,(R-Z))+Z
                dprft_dR = np.multiply(W,P_in)
                dprft_dZ = np.multiply(W,(1-P_in))
                
                dprft_dWdW = 0
                dprft_dWdR = P_in
                dprft_dWdZ = 1-P_in
                dprft_dRdR = 0
                dprft_dRdZ = 0
                dprft_dZdZ = 0
                
                #return list
                """
                Note that this function returns a dictionary
                """
                c={'dWdW': dprft_dWdW,'dWdR': dprft_dRdR,'dWdZ': dprft_dWdZ,
                'dRdR': dprft_dRdR,'dRdZ': dprft_dRdZ,'dZdZ': dprft_dZdZ }
                return c
            
            
            def g_VaR_hsn(params):
                
                W = params.W
                R = params.R
                Z = params.Z
                S_W =np.multiply( Std,W)
                S_RZ = np.multiply((R-Z),Std)
                W_RZ_S =np.multiply( W,S_RZ)
                O_W_RZ_S = corr.mtx*(W_RZ_S)
                W_RZ_S_O = (W_RZ_S).T*corr.mtx
                
                #calculate scaling factor and variance
                scale.fct = sqrt((1-eps)/eps)
                Var = (W_RZ_S).T*corr.mtx*(W_RZ_S)
                #calculate derivatives of Var
                dVar_dW =  np.multiply(O_W_RZ_S,S_RZ)#*2
                dVar_dR =  np.multiply(O_W_RZ_S,S_W) #*2
                dVar_dZ = np.multiply(-O_W_RZ_S,S_W) #*2
                
                
                dVaR_dWdW =  Var[0,0]^(-1/2)*(np.diag(S_RZ)*corr_mtx*np.diag(S_RZ))-Var[0,0]^(-3/2)*dVar_dW*(W_RZ_S_O*(S_RZ).T)
                dVaR_dWdR =  Var[0,0]^(-1/2)*((np.diag(S_RZ)*corr_mtx*np.diag(S_W))+np.diag(O_W_RZ_S*Std))-Var[0,0]^(-3/2)*dVar_dW*(W_RZ_S_O*(S_W).T)
                dVaR_dWdZ = -Var[0,0]^(-1/2)*((np.diag(S_RZ)*corr_mtx*np.diag(S_W))+diag(O_W_RZ_S*Std))+Var[0,0]^(-3/2)*dVar_dW*(W_RZ_S_O*(S_W).T)
                dVaR_dRdR =  Var[0,0]^(-1/2)*(np.diag(S_W)*corr_mtx*np.diag(S_W))-Var[0,0]^(-3/2)*dVar_dR*(W_RZ_S_O*(S_W).T)
                dVaR_dRdZ = -Var[0,0]^(-1/2)*(np.diag(S_W)*corr_mtx*np.diag(S_W))+Var[0,0]^(-3/2)*dVar_dR*(W_RZ_S_O*(S_W).T)
                dVaR_dZdZ = -Var[0,0]^(-1/2)*(np.diag(S_W)*corr_mtx*np.diag(S_W))+Var[0,0]^(-3/2)*dVar_dR*(W_RZ_S_O*(S_W).T)

                #Return Dict
                d={'dWdW': dVaR_dWdW*scale_fct,'dWdR':dVaR_dRdR*scale_fct,'dWdZ':dVaR_dWdZ*scale_fct,
                'dRdR': dVaR_dRdR*scale_fct,'dRdZ': dVaR_dRdZ*scale_fct,'dZdZ':dVaR_dZdZ*scale_fct}
                 
                return d
            """
            We Now define constraints on optimization
            
            """
            ###Constraint 1: Amt Bought + (Amt Sold)*(Collateral on Amt Sold) <= Amt in Trust Channel
            def cnst1_val(params):
                W = params.W
                R = params.R
                Z = params.Z
                val = ifelse(W_TF,sum(W[W_optim]),0)+ifelse(S_TF,np.multiply(W[S_optim],(-Z[S_optim])),0)
                return val
            
            def cnst1_grd(params):
                W = params.W
                R = params.R
                Z = params.Z
                dW_dr = params.dW_dr
                dW_dz = params.dW_dz
                
                dW=np.ones(W_optim)
                """
                The function below returns either a zero or some single valuse and not a vector
                """
                dRs = ifelse(S_TF,-Z[S_optim]*dW_dr[S_optim],0)
                dZs = ifelse(S_TF,W[S_optim]+(-Z[S_optim])*dW_dz[S_optim],0)
                k={'dW':dW,'dRs':dRs,'dZs':dZs}
                return k
            #Constraint 2: Amt Bought+Amt Novated = Prespecified Value (only used for frwd-prop)
            def cnst2_val(params):
                W = params.W
                R = params.R
                Z = params.Z
                val = ifelse(W_TF,sum(W[W_optim]),0)+ ifelse(S_TF,W[S_optim],0)-ifelse(C_TF,Clim,0)
                return(val*C_TF)
            
            def cnst2_grd(params):
                
                W = params.W
                R = params.R
                Z = params.Z
                dW_dr = params.dW_dr
                dW_dz = params.dW_dz
                dW = np.zeros(W.optim)
                dW[(len(dW)-n_C):length(dW)] = C_TF
                dRs = ifelse(S_TF*C_TF,dW_dr[S_optim],0) #0 unless S.TF and C.TF
                dZs = ifelse(S_TF*C_TF,dW_dz[S_optim],0)
                m={'dW':dW,'dRs':dRs,'dZs':dZs}
                return m
            """
             Piece together above functions into optimizer inputs
            """ 
            #For optimizer to fetch gradient and objective in combined list
            def obj_val_grd(X):
                params = opt_update_WRZ(X)
                obj = obj(params)
                grad =np.zeros(n_opt)
                if(W_TF):
                    grad[W_indx] = d_obj_dw(params)
                if(R_TF):
                    grad[R_indx] = d_obj_dr(params)
                if(Z_TF):
                    grad[Z_indx] = d_obj_dz(params)
                t2={'objective':obj,'gradient':grad}
                
            #For optimizer to fetch inequality constraint graident and value in combined list
            def cnst_ineq_val_grd(X):
                params = opt_update_WRZ(X)
                c_val = cnst1_val(params)
                c_grd = cnst1_grd(params)
                grad = np.zeros([1,n_opt])
                #assemble jacobian
                grad[1,W_indx] = c_grd.dW
                if(S.TF):
                    grad[1,RS_indx] = c_grd.dRs
                    grad[1,ZS_indx] = c_grd.dZs
                t3={'constraints':c.val,'jacobian':grad}

            #For optimizer to fetch equality constraint graident and value in combined list
            def cnst_eq_val_grd(X):
                params = opt_update_WRZ(X)
                #update parameters
                c_val = cnst2_val(params)
                c_grd = cnst2_grd(params)
                #assemble jacobian
                grad = np.zeros([1,n_opt])
                grad[1,W_indx] = c_grd.dW
                if(S_TF):
                    grad[1,RS_indx] = c_grd.dRs
                    grad[1,ZS_indx] = c_grd.dZs
                k3={'constraints':c_val,'jacobian':grad}
                return k3
            #In advent equality and inequality constraints need be combined
            def cnst_cmb_val_grd(X):
                #update parameters
                params = opt_update_WRZ(X)
                indx = [True,Cash_TF,C_TF,Cash_TF,C_TF]
                #Inequality Value
                val_1 = cnst1_val(params)
                #Equality Values
                val_2 = cnst2_val(params)
                val_3 = cnst3_val(params)
                #Equality values Reversed (for opposite inequality)
                val_4 = -val_2
                val_5 = -val_3
                #assemble
                val = [val_1,val_2,val_3,val_4,val_5]
                #Inequality Gradient
                grd_1 = cnst1_grd(params)
                grd_2 = cnst2_grd(params)
                grd_3 = cnst3_grd(params)
                #Gradient values Reversed (for opposite inequality)
                """
                Does this imply recursion
                #Gradient values Reversed (for opposite inequality)
                grd.4 <- lapply(grd.2,function(x) -x)
                grd.5 <- lapply(grd.3,function(x) -x)
                """
                grd_4=-grd_2
                grd_5=-grd_3
                grad=np.zeros([5,n.opt])
                grad[1,W_indx] = grd_1.dW
                grad[2,W_indx] = grd_2.dW
                grad[3,W_indx] = grd_3.dW
                grad[4,W_indx] = grd_3.dW
                grad[5,W_indx] = grd_4.dW
                if(S.TF):
                    grad[1,RS_indx] = grd_1.dR
                    grad[1,ZS_indx] = grd_1.dZ
                    grad[2,RS_indx] = grd_2.dR
                    grad[2,ZS_indx] = grd_2.dZ
                    grad[3,RS_indx] = grd_3.dR
                    grad[3,ZS_indx] = grd_3.dZ
                    grad[4,RS_indx] = grd_4.dR
                    grad[4,ZS_indx] = grd_4.dZ
                    grad[5,RS_indx] = grd_5.dR
                    grad[5,ZS_indx] = grd_5.dZ
                k6={'constraints':val[indx],'jacobian':grad[indx]}
                return k6
            
            #### Set initial varibles ####
            #create starting values, upper bound, lower bound, and solve for respective indices
            X0_opt = [pd.NA]* n_opt
            LB_opt = [pd.NA]* n_opt
            UB_opt = [pd.NA]* n_opt
            W_indx = pd.NA
            R_indx = pd.NA
            Z_indx = pd.NA
            RS_indx = pd.NA
            ZS_indx = pd.NA
            
            def init_val_opt():
                global X0_opt,LB_opt,UB_opt,W_indx,R_indx,Z_indx,RS_indx,ZS_indx
                if(W.TF):
                    W_indx=len(W_optim)
                    LB_opt[0:W_indx]=0.01+S_TF*Wlim*0.15*0.2
                    UB_opt[0:W_indx]=Wlim
                    
            
            
            
            """
            All these needed to be passsed into the tps algorithm directly
             #### Pass into Optimizer ####
            The thin plate spline has been implmented
            Previously
            The inputs from this part to be sent to it
            """
            
            
                
            

                        
                                    
                      
                
        
                
        
                    
                    
                
                
                
                
                
            
            
        

In [127]:
w=[1,3]
#np.expand_dims(w,axis=0)
#w2=w[:,np.newaxis]

#c=np.ones([3,2])
#np.shape(w*c)[0]

In [118]:
z=pd.NA

In [119]:
optim_kumar()


In [52]:
k=[1,2,3,5]
if(np.count_nonzero(k)==len(k)):
    print('yes')
    
sum(k[:3])

yes


6

In [144]:
c=[4,5]
type(c) is list

True

In [145]:
a=[3,3]
a=[7]+a

In [146]:
a

[7, 3, 3]

In [164]:
c=[[7,3],[8,9],[9,9]]
k=np.shape(c)
m=np.zeros(k[1])
d=np.insert(c,[0],m,axis=0)
l=np.zeros(k[0]+1)
l=np.transpose(l)
d=np.insert(d,0,l,axis=1)
k=np.shape(d)

In [172]:
np.where(d==0)[0]


array([0, 0, 0, 1, 2, 3])

In [158]:
np.isnan([2,3,1,0]).any()

False

In [163]:
x=[2,3,4,5,9]
np.all(x)

True

In [32]:
c=[[0,1,2,3],[4,9,0,0]]
k=np.array(c)
t=np.where(k==0)[0]
t[2]
len(np.shape(c))

2

In [29]:
c=[1,3,4]
d=[4,5,6,6]


In [36]:
p=np.zeros([3,2])
for j in range(2):
    p[:,j]=[1,2,3]

In [37]:
p

array([[1., 1.],
       [2., 2.],
       [3., 3.]])

In [38]:
p=[[2],[3,4],5]

In [41]:
np.shape(p)

(3,)

In [71]:
np.sqrt(9)


3.0

In [90]:
c={"sid":2}
c["sid"]

2

In [128]:
ifelse(False,2,4)

4

In [160]:
k=[pd.NA]*5

In [161]:
k

[<NA>, <NA>, <NA>, <NA>, <NA>]